# MNE(操作指南)

### 文件信息

In [1]:
# 导入库文件
import mne
import matplotlib

In [10]:
# 文件路径
path = 'F:/CHB_MIT/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01_03.edf'
# 交互式绘图
matplotlib.use('Qt5Agg')
# 读文件
raw = mne.io.read_raw_edf(path)
# 可视化数据
raw.plot()

Extracting EDF parameters from F:\CHB_MIT\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\one\AppData\Local\Temp\ipykernel_28584\3332629531.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(path)


<MNEBrowseFigure size 800x800 with 4 Axes>

In [14]:
# 可视化电极位置
from mne.channels import make_standard_montage

print(raw.info['dig'])  # 查看电极坐标信息
print(raw.info['chs'][0]['loc'])  # 检查单个电极的坐标
# raw.plot_sensors(ch_type='eeg',show_names=True)

[<DigPoint |        LPA : (-82.5, 0.0, 0.0) mm      : head frame>, <DigPoint |     Nasion : (-0.0, 114.0, 0.0) mm     : head frame>, <DigPoint |        RPA : (82.5, -0.0, -0.0) mm     : head frame>]
[nan nan nan nan nan nan nan nan nan nan nan nan]


C:\Users\one\AppData\Local\Temp\ipykernel_28584\2720303542.py:4: RuntimeWarning: DigMontage is only a subset of info. There are 23 channel positions not present in the DigMontage. The channels missing from the montage are:

['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw.set_montage(montage, on_missing='warn')  # 应用到数据


In [ ]:
# 定位电极位置
raw.set_montage(montage='standard_1020', on_missing='ignore')

# 查看坏导
raw.info['bads']

# 标记坏导
# raw.info['bads'] = ['C1', 'T7', 'Temp']

In [6]:
# 数据读入内存
raw.load_data()

# 插值处理坏导
# raw.interpolate_bads(exclude=['temp'])

Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


<RawEDF | chb01_03.edf, 23 x 921600 (3600.0 s), ~161.7 MiB, data loaded>

In [7]:
# 显示功率谱密度
raw.plot_psd()

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 8.000 (s)
Plotting power spectral density (dB=True).


C:\Users\one\AppData\Local\Temp\ipykernel_28584\3196231698.py:2: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  raw.plot_psd()


<MNELineFigure size 1000x350 with 1 Axes>

In [8]:
# 滤波
raw.filter(l_freq=0.1, h_freq=40)
raw.notch_filter(freqs=50)  # 工频干扰
raw.plot_psd()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 8449 samples (33.004 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 8.000 (s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Plotting power spectral density (dB=True).


C:\Users\one\AppData\Local\Temp\ipykernel_28584\3627462913.py:4: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  raw.plot_psd()


<MNELineFigure size 1000x350 with 1 Axes>

In [ ]:
# 重参考
# raw.set_eeg_reference()

In [9]:
# ICA（独立成分分析）
ica = mne.preprocessing.ICA(n_components=10)
ica.fit(raw)
# 可视化每个独立成分的头皮分布
ica.plot_components()

Fitting ICA to data using 23 channels (please be patient, this may take a while)
Selecting by number: 10 components
Fitting ICA took 7.1s.


RuntimeError: No digitization points found.

In [ ]:
# 绘制每个独立成分的头皮分布
pick_id = [0,1,2,3,4,5,6,7,8,9]
ica.plot_properties(raw, picks=pick_id)

In [ ]:
# 剔除一些成分
exclude_id = [0,2,4,6,8]
ica.plot_overlay(raw, exclude=exclude_id)
# 剔除
ica.exclude = exclude_id